In [1]:
#TODO J: Notebook can be deleted
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import lpips
import os
import imageio
import numpy as np

from utils.metrics import MSE, PSNR, SSIM, LPIPS, RMSE

In [19]:
def read_images_in_dir(imgs_dir):
    imgs = []
    fnames = os.listdir(imgs_dir)
    fnames.sort()
    for fname in fnames:
        img_path = os.path.join(imgs_dir, fname)
        img = imageio.imread(img_path)
        img = (np.array(img) / 255.).astype(np.float32)
        img = np.transpose(img, (2, 0, 1))
        imgs.append(img)
    
    imgs = np.stack(imgs)       
    return imgs

def read_depth_in_dir(depth_dir):
    depths = []
    fnames = os.listdir(depth_dir)
    fnames.sort()

    for fname in fnames:            
        depth_path = os.path.join(depth_dir, fname)
        depth = imageio.imread(depth_path)
        depth = (depth).astype(np.float32)
        depths.append(depth)
    
    depths = np.stack(depths)       
    return depths

def estim_error(estim, gt, estim_depth, gt_depth):
    errors = dict()
    metric = MSE()
    errors["mse"] = metric(estim, gt).item()
    metric = PSNR()
    errors["psnr"] = metric(estim, gt).item()
    metric = SSIM()
    errors["ssim"] = metric(estim, gt).item()
    metric = LPIPS()
    errors["lpips"] = metric(estim, gt).item()
    metric = RMSE()
    errors["depth_rmse"] = metric(estim_depth, gt_depth).item()
    print(estim_depth)
    print(gt_depth)
    
    return errors

def save_error(errors, save_dir):
    save_path = os.path.join(save_dir, "metrics.txt")
    f = open(save_path,"w")
    f.write( str(errors) )
    f.close()

In [20]:
files_dir = "./logs/report_johannes_11_gnll_even_smaller_weight/renderonly_test_399999/"

estim_dir = os.path.join(files_dir, "estim")
gt_dir = os.path.join(files_dir, "gt")

estim_depth_dir = os.path.join(files_dir, "estim_depth")
gt_depth_dir = os.path.join(files_dir, "gt_depth")

estim = read_images_in_dir(estim_dir)
gt = read_images_in_dir(gt_dir)

estim_depth = read_depth_in_dir(estim_depth_dir)
gt_depth = read_depth_in_dir(gt_depth_dir)

estim = torch.Tensor(estim).cuda()
gt = torch.Tensor(gt).cuda()
estim_depth = torch.Tensor(estim_depth).cuda()
gt_depth = torch.Tensor(gt_depth).cuda()

errors = estim_error(estim, gt, estim_depth, gt_depth)
save_error(errors, files_dir)
print(errors)

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /home/kirwul/miniconda3/envs/dnerf/lib/python3.6/site-packages/lpips/weights/v0.1/vgg.pth
tensor([[[119., 119., 120.,  ..., 176., 176., 176.],
         [121., 120., 120.,  ..., 176., 176., 176.],
         [122., 122., 121.,  ..., 176., 176., 176.],
         ...,
         [ 32.,  34.,  37.,  ...,  80.,  81.,  81.],
         [ 35.,  36.,  40.,  ...,  80.,  80.,  80.],
         [ 36.,  38.,  40.,  ...,  80.,  80.,  80.]],

        [[110., 114., 117.,  ..., 172., 172., 172.],
         [108., 110., 113.,  ..., 172., 172., 172.],
         [106., 108., 110.,  ..., 173., 173., 173.],
         ...,
         [ 52.,  51.,  51.,  ...,  84.,  84.,  85.],
         [ 54.,  51.,  51.,  ...,  83.,  84.,  84.],
         [ 53.,  54.,  52.,  ...,  82.,  83.,  83.]],

        [[122., 122., 124.,  ..., 173., 173., 174.],
         [121., 122., 122.,  ..., 173., 173., 174.],
         [121., 121., 121.,  ..., 173., 174.,